In [1]:
import numpy as np

from RushHour4.core import Map, Environment
from RushHour4.interact import Game

In [2]:
map = Map(10, 10)
map.view()

o,o,o,o,o,o,o,o,o,o
o,o,o,o,o,o,o,o,o,o
o,o,o,o,o,o,o,o,o,o
o,o,o,o,o,o,o,o,o,o
o,o,o,o,o,o,o,o,o,o
o,o,o,o,o,o,o,o,o,o
o,o,o,o,o,o,o,o,o,o
o,o,o,o,o,o,o,o,o,o
o,o,o,o,o,o,o,o,o,o
o,o,o,o,o,o,o,o,o,o


In [3]:
game = Game(map, 50)

In [6]:
print(game)

o o o o o o o o o o
o o o o o o o o o o
o o o o o o o o o o
o o o o o o o o o o
o o o o o o o o o o
o o o o o o o o o o
o o o o o o o o o o
o o o o o o o o o o
o o o o o o o o o o
o o o o o o o o o o
